<a href="https://colab.research.google.com/github/nurchamidah/Self-Project/blob/main/Precipitation_Data_Processing/Precip_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data Set

## Import Library

In [2]:
import pandas as pd
import math
import numpy as np

## Read Dataset

In [3]:
# Read dataset from Google Sheet (use raw link)
sheet_url = 'https://docs.google.com/spreadsheets/d/14PP1x4P2QmyQwdHSdJDxjcGjFP_G8qwV/edit#gid=505961197'

sheet_url_trf = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
print(sheet_url_trf)

df = pd.read_csv(sheet_url_trf)
df.head()

https://docs.google.com/spreadsheets/d/14PP1x4P2QmyQwdHSdJDxjcGjFP_G8qwV/export?format=csv&gid=505961197


,date,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,weatherIconUrl,...,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex
0,2013-01-01,24,29,85,4,7,169,SSE,356,[{'value': 'https://cdn.worldweatheronline.com...,...,88,24,74,27,81,8,13,31,88,6
1,2013-01-02,24,30,86,6,9,162,SSE,176,[{'value': 'https://cdn.worldweatheronline.com...,...,88,24,75,28,82,9,14,31,88,6
2,2013-01-03,24,29,85,4,6,225,SW,356,[{'value': 'https://cdn.worldweatheronline.com...,...,88,24,75,27,81,7,11,31,88,6
3,2013-01-04,24,30,87,5,9,159,SSE,299,[{'value': 'https://cdn.worldweatheronline.com...,...,89,24,75,27,81,8,13,32,89,6
4,2013-01-05,24,29,83,4,6,125,SE,299,[{'value': 'https://cdn.worldweatheronline.com...,...,87,24,75,27,81,6,10,31,87,6


## Check Data Type

In [4]:
# Check the table, from the data type, row, and empty
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4017 entries, 0 to 4016
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             4017 non-null   object
 1   time             4017 non-null   int64 
 2   tempC            4017 non-null   int64 
 3   tempF            4017 non-null   int64 
 4   windspeedMiles   4017 non-null   int64 
 5   windspeedKmph    4017 non-null   int64 
 6   winddirDegree    4017 non-null   int64 
 7   winddir16Point   4017 non-null   object
 8   weatherCode      4017 non-null   int64 
 9   weatherIconUrl   4017 non-null   object
 10  weatherDesc      4017 non-null   object
 11  precipMM         4017 non-null   object
 12  precipInches     4017 non-null   object
 13  humidity         4017 non-null   int64 
 14  visibility       4017 non-null   int64 
 15  visibilityMiles  4017 non-null   int64 
 16  pressure         4017 non-null   int64 
 17  pressureInches   4017 non-null   

## Change Data Type

Change type of date data into date time and precipeMM data into float.

In [5]:
# Ensure precipMM is of type string
df['precipMM'] = df['precipMM'].astype(str)

# Replace commas with dots and convert to float
df['precipMM'] = df['precipMM'].str.replace(',', '.').astype(float)

# Change date type
df = df.copy()
df['date'] = pd.to_datetime(df['date'])

print(df.dtypes)
print(df)

date               datetime64[ns]
time                        int64
tempC                       int64
tempF                       int64
windspeedMiles              int64
windspeedKmph               int64
winddirDegree               int64
winddir16Point             object
weatherCode                 int64
weatherIconUrl             object
weatherDesc                object
precipMM                  float64
precipInches               object
humidity                    int64
visibility                  int64
visibilityMiles             int64
pressure                    int64
pressureInches              int64
cloudcover                  int64
HeatIndexC                  int64
HeatIndexF                  int64
DewPointC                   int64
DewPointF                   int64
WindChillC                  int64
WindChillF                  int64
WindGustMiles               int64
WindGustKmph                int64
FeelsLikeC                  int64
FeelsLikeF                  int64
uvIndex       

# EDA

## Get Date and Precipitation each Day

From the entire table, take the date and recipeMM data for further processing.

In [7]:
# Take the date and precipitation columns
df_precip = df[['date', 'precipMM']]

# Tampilkan data yang dipilih
df_precip

,date,precipMM
0,2013-01-01,31.9
1,2013-01-02,6.0
2,2013-01-03,6.1
3,2013-01-04,16.5
4,2013-01-05,19.7
...,...,...
4012,2023-12-27,8.8
4013,2023-12-28,3.5
4014,2023-12-29,5.9
4015,2023-12-30,3.4


## Get Maximum Precipitation each Month

Create a table containing the maximum daily rainfall for each month.

In [53]:
# Add year and month columns using .loc to avoid SettingWithCopyWarning
df_precip.loc[:, 'tahun'] = df_precip[date_column].dt.year
df_precip.loc[:, 'bulan'] = df_precip[date_column].dt.month

# Calculate the maximum daily rainfall each month
df_max_harian_bulanan = df_precip.groupby(['tahun', 'bulan'])['precipMM'].max().reset_index()

# Create a pivot table
df_pivot = df_max_harian_bulanan.pivot(index='tahun', columns='bulan', values='precipMM')

# Change the name of the month column to the name of the month in Indonesian
bulan_mapping = {
    1: 'Januari', 2: 'Februari', 3: 'Maret', 4: 'April', 5: 'Mei', 6: 'Juni',
    7: 'Juli', 8: 'Agustus', 9: 'September', 10: 'Oktober', 11: 'November', 12: 'Desember'
}
df_pivot.columns = [bulan_mapping[col] for col in df_pivot.columns]

print(df_pivot)

       Januari  Februari  Maret  April   Mei  Juni   Juli  Agustus  September  \
tahun                                                                           
2013      37.3      44.6   89.8   25.0  24.8  33.8   31.4     25.0       25.6   
2014      34.2      22.9   23.6   51.5  23.1  16.1   34.5     31.8       54.5   
2015      82.4      31.0   31.2   16.1  23.2  26.8   27.9     24.6       26.7   
2016      31.0      23.5   28.8   15.6  26.1  29.7   25.2     41.6       38.6   
2017      25.4      40.2   30.4   27.0  30.3  47.6  103.4     21.6       32.3   
2018      31.1      34.3   23.1   21.9  31.0  24.0   25.5     38.6       35.9   
2019      28.8      12.5   33.7   33.8  20.7  18.7   18.4     17.6       15.2   
2020      50.9      63.4   23.7   26.4  18.4  31.3   31.8     29.3       27.5   
2021      51.5      25.1   46.7   59.3  93.6  37.0   24.9     17.5       25.0   
2022      59.1      41.4   85.2   40.6  30.3  26.3   57.8     25.9       40.0   
2023      36.7      17.7   3

If there is a NaN value use code below to change NaN into 0

df_pivot.fillna(0, inplace=True)

## Average Daily Rainfall

Create a table of average daily rainfall for each month.

---



In [54]:
# Add year and month columns using .loc to avoid SettingWithCopyWarning
df_precip.loc[:, 'tahun'] = df_precip[date_column].dt.year
df_precip.loc[:, 'bulan'] = df_precip[date_column].dt.month

# Calculate the average daily rainfall each month
df_avg_harian_bulanan = df_precip.groupby(['tahun', 'bulan'])['precipMM'].mean().reset_index()

# Divide the average result by 24
df_avg_harian_bulanan['precipMM'] = df_avg_harian_bulanan['precipMM'] / 24

# Create a pivot table
df_pivot = df_avg_harian_bulanan.pivot(index='tahun', columns='bulan', values='precipMM')

# Change the name of the month column to the name of the month in Indonesian
bulan_mapping = {
    1: 'Januari', 2: 'Februari', 3: 'Maret', 4: 'April', 5: 'Mei', 6: 'Juni',
    7: 'Juli', 8: 'Agustus', 9: 'September', 10: 'Oktober', 11: 'November', 12: 'Desember'
}
df_pivot.columns = [bulan_mapping[col] for col in df_pivot.columns]

print(df_pivot)

        Januari  Februari     Maret     April       Mei      Juni      Juli  \
tahun                                                                         
2013   0.413575  0.514435  0.344489  0.496250  0.423656  0.420278  0.330780   
2014   0.457392  0.277530  0.292070  0.466806  0.308602  0.320278  0.510081   
2015   0.658065  0.252232  0.272043  0.231944  0.425941  0.333194  0.144355   
2016   0.177016  0.228161  0.222984  0.158472  0.371640  0.387639  0.428226   
2017   0.406989  0.425595  0.473253  0.461667  0.516935  0.503750  0.611962   
2018   0.532527  0.344792  0.325000  0.342361  0.378226  0.321528  0.378495   
2019   0.337097  0.145387  0.309812  0.267639  0.306183  0.398056  0.243683   
2020   0.315995  0.280603  0.303898  0.253750  0.242070  0.391944  0.395565   
2021   0.604973  0.403869  0.494086  0.381389  0.533468  0.339028  0.286290   
2022   0.506586  0.370238  0.548387  0.476250  0.455108  0.251944  0.514516   
2023   0.392473  0.296875  0.518683  0.455000  0.629

## Precipitation Max / Year


Create a table of maximum rainfall each year from the maximum rainfall each month.

In [55]:
# Tambahkan kolom 'tahun' untuk mengelompokkan berdasarkan tahun
df['tahun'] = df['date'].dt.year

# Menghitung curah hujan maksimum per tahun
df_max_per_year = df.groupby('tahun')['precipMM'].max().reset_index()
df_max_per_year.rename(columns={'precipMM': 'curah_hujan_maksimum'}, inplace=True)

# Menghitung rata-rata curah hujan maksimum
average_max_rainfall = df_max_per_year['curah_hujan_maksimum'].mean()

print(df_max_per_year)

    tahun  curah_hujan_maksimum
0    2013                  89.8
1    2014                  58.1
2    2015                  82.4
3    2016                  42.1
4    2017                 103.4
5    2018                  45.2
6    2019                  85.6
7    2020                 170.8
8    2021                  93.6
9    2022                  85.2
10   2023                  42.1


## Planned Rainfall and Rainfall Intensity



Calculate average daily rainfall/hour

In [56]:
# Calculate the average daily/hourly rainfall
df_pivot.mean()
df_pivot_avg = df_pivot.values.mean()
df_pivot_avg

0.40163037560840675

In [57]:
# Add new column
df_max_per_year['n'] = len(df_max_per_year['curah_hujan_maksimum'])
df_max_per_year['curah_hujan_avg'] = df_max_per_year['curah_hujan_maksimum'].sum() / df_max_per_year['n']
df_max_per_year['(x-x1)^2'] = (df_max_per_year['curah_hujan_maksimum'] - df_max_per_year['curah_hujan_avg']) ** 2
df_max_per_year['n'] = len(df_max_per_year['curah_hujan_maksimum'])
df_max_per_year['m'] = df_max_per_year['curah_hujan_maksimum'].rank(ascending=False, method='min')
df_max_per_year['Yn'] = df_max_per_year.apply(lambda row: -math.log(-math.log(((row['n'] + 1) - row['m']) / (row['n'] + 1))), axis=1)
df_max_per_year['Yn_avg'] = df_max_per_year['Yn'].sum() / df_max_per_year['n']
df_max_per_year['(Yn-Yn_avg)^2'] = (df_max_per_year['Yn'] - df_max_per_year['Yn_avg']) ** 2
df_max_per_year['SD'] = math.sqrt((df_max_per_year['(x-x1)^2'].sum())/ (df_max_per_year['n'].mean() - 1))
df_max_per_year['SN'] = math.sqrt(df_max_per_year['(Yn-Yn_avg)^2'].sum() / (df_max_per_year['n'].mean() - 1))
df_max_per_year['PUH'] = range(1,12)
df_max_per_year['Yt'] = df_max_per_year.apply(lambda row: -math.log(-math.log((row['PUH'] - 1) / row['PUH'])) if row['PUH'] > 1 else float(0.0), axis=1)
df_max_per_year['Yt-Yn_avg'] = df_max_per_year['Yt'] - df_max_per_year['Yn_avg']
df_max_per_year['k'] = ((df_max_per_year['Yt'] - df_max_per_year['Yn'].mean()) / df_max_per_year['SN'].shift(1)).fillna(0.0)
df_max_per_year['Xt'] = df_max_per_year['curah_hujan_avg'] + (df_max_per_year['SD'] * df_max_per_year['k'].fillna(0.0))
df_max_per_year['Risiko Hidrologi'] = (1 - (1 - (1 / df_max_per_year['PUH'])) ** 5) * 100
df_max_per_year['Intensitas Curah Hujan Rencana (I)'] = (df_max_per_year['Xt']/24) * ((24/df_pivot_avg) ** 0.67)

df_max_per_year

,tahun,curah_hujan_maksimum,n,curah_hujan_avg,(x-x1)^2,m,Yn,Yn_avg,(Yn-Yn_avg)^2,SD,SN,PUH,Yt,Yt-Yn_avg,k,Xt,Risiko Hidrologi,Intensitas Curah Hujan Rencana (I)
0,2013,89.8,11,81.663636,66.200413,4.0,0.902720,0.529345,0.139409,36.973484,0.973314,1,0.000000,-0.529345,0.000000,81.663636,100.000000,52.722405
1,2014,58.1,11,81.663636,555.244959,8.0,-0.094048,0.529345,0.388619,36.973484,0.973314,2,0.366513,-0.162832,-0.167297,75.478096,96.875000,48.728993
2,2015,82.4,11,81.663636,0.542231,7.0,0.132996,0.529345,0.157093,36.973484,0.973314,3,0.902720,0.373375,0.383612,95.847123,86.831276,61.879327
3,2016,42.1,11,81.663636,1565.281322,10.0,-0.583198,0.529345,1.237752,36.973484,0.973314,4,1.245899,0.716554,0.736200,108.883531,76.269531,70.295690
4,2017,103.4,11,81.663636,472.469504,2.0,1.701983,0.529345,1.375080,36.973484,0.973314,5,1.499940,0.970595,0.997206,118.533826,67.232000,76.525963
5,2018,45.2,11,81.663636,1329.596777,9.0,-0.326634,0.529345,0.732701,36.973484,0.973314,6,1.701983,1.172638,1.204789,126.208890,59.812243,81.481018
6,2019,85.6,11,81.663636,15.494959,5.0,0.618046,0.529345,0.007868,36.973484,0.973314,7,1.869825,1.340480,1.377232,132.584715,53.733563,85.597279
7,2020,170.8,11,81.663636,7945.291322,1.0,2.441716,0.529345,3.657164,36.973484,0.973314,8,2.013419,1.484074,1.524763,138.039449,48.709106,89.118879
8,2021,93.6,11,81.663636,142.476777,3.0,1.245899,0.529345,0.513450,36.973484,0.973314,9,2.138911,1.609566,1.653696,142.806553,44.507104,92.196543
9,2022,85.2,11,81.663636,12.505868,6.0,0.366513,0.529345,0.026514,36.973484,0.973314,10,2.250367,1.721022,1.768209,147.040466,40.951000,94.929976


If you use different file, make sure to change the value of PUH. The formula for the PUH is = range(1, year_use+1). So if u use 11 years, the formula will be  = range(1, 11+1) = range (1,12)

## Intensitas Curah Hujan

The rainfall intensity was chosen during the 3-year PUH with a hydrological risk of more than 85% with a planned rainfall intensity of 61.87 mm/hour and a planned daily rainfall of 108.88 mm/hour.

# Export Table

In [230]:
from google.colab import files

# Ekspor ke CSV
df_max_per_year.to_csv('CH Bulungan.csv', index=False)
files.download('CH Bulungan.csv')

# Ekspor ke Excel
df.to_excel('CH Bulungan.xlsx', index=False)
files.download('CH Bulungan.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>